In [6]:
#imports and reading in files + path setup
import os
import re
import pandas as pd
flatfiles=[]
for filename in os.listdir("genomes"):
    flatfiles.append(filename)

path='C:\\Users\\Matt\\Desktop\\Bioinfo Project\\genomes\\'

In [7]:
def parseFile(filename, path):
    fileString=open(path+filename).read()
    #AQUIRE START INDICIES
    startIndex=fileString.find("CDS")
    starts=[]
    while startIndex>0:
        temp=fileString[startIndex:startIndex+50]
        ex=r'[ (]\d+'
        match=re.search(ex,temp)
        try:
            starts.append(int(re.search(ex,temp).group(0)[1:]))
        except:
            pass
        #print(filename)
        startIndex=fileString.find("CDS",startIndex+1)
    #print(starts)
    
    #ISOLATE GENOME
    genome=""
    for char in fileString[fileString.find("ORIGIN"):len(fileString)]:
        if char=='t'or char=='g' or char=='a' or char=='c':
            genome+=char
    genome=fileString[fileString.find("ORIGIN")+6:len(fileString)]
    genome=re.sub(r'[^a-z]+',"",genome)
    
    #CHANGE T BASE TO U TO REPRESENT AS RNA
    genomeRNA=""
    for char in genome:
        if char=='t':
            genomeRNA+='u'
        else:
            genomeRNA+=char
    
    #Get all start codons in genome
    startCodonIndicies=[]
    for i in range(10,len(genomeRNA)):
        #print(genome[i:i+3])
        if genomeRNA[i:i+3]=='aug':
            startCodonIndicies.append(i+1)
    
    falsePositiveStarts=[]
    for index in startCodonIndicies:
        if index in starts:
            pass
        else:
            falsePositiveStarts.append(index)
    
    trueStarts=[]
    for index in starts:
        code=genomeRNA[index-11:index+14]+filename
        if code[10:13]!="aug": 
            print(code)
            print(len(genome))
            print(len(genomeRNA))
            print(index)
            print(starts)
        trueStarts.append(genomeRNA[index-11:index+14]+filename+ " " + str(index))
        break
    falseStarts=[]
    for index in falsePositiveStarts:
        falseStarts.append(genomeRNA[index-11:index+14]+filename+ " " + str(index))
    
    return {"trueStarts": trueStarts, "falseStarts": falseStarts}

In [8]:
#develop scoring matrix


#load true and false start spaces
trueStartSpace=[]
falseStartSpace=[]
#first 10 of flat files
for i in range(len(flatfiles)):
    if 'A'=='A':
        spaces=parseFile(flatfiles[i],path)
        trueStartSpace+=spaces["trueStarts"]
        falseStartSpace+=spaces["falseStarts"]

numNucs=25
#build consensus matrices
print("True start codon identities by position")
startIdentities={"A": [0 for i in range(numNucs)], "U": [0 for i in range(numNucs)], "G": [0 for i in range(numNucs)], "C": [0 for i in range(numNucs)]}
for i in trueStartSpace:
    temp=i.replace(" ","").upper()
    for n in range(numNucs):
        try:
            startIdentities[temp[n]][n]+=1
        except:
            print("except: " + temp)
#make data frame of start identities for clean format of output
startIDsDF=pd.DataFrame.from_dict(startIdentities)
print(startIDsDF)

print("\nTrue start codon frequency by position")
for key in startIdentities:
    for i in range(numNucs):
        startIdentities[key][i]=round(startIdentities[key][i]/len(trueStartSpace),2)
startFreqsDF=pd.DataFrame.from_dict(startIdentities)
print(startFreqsDF.T)

print("\nFalse start codon identities by position")
falseStartIdentities={"A": [0 for i in range(numNucs)], "U": [0 for i in range(numNucs)], "G": [0 for i in range(numNucs)], "C": [0 for i in range(numNucs)]}
for i in falseStartSpace:
    temp=i.replace(" ","").upper()
    try:
        for n in range(numNucs):
            falseStartIdentities[temp[n]][n]+=1
    except:
        pass
#make data frame of false start identities for clean format of output
falseStartIDsDF=pd.DataFrame.from_dict(falseStartIdentities)
print(falseStartIDsDF)

print("\nFalse start codon frequency by position")
for key in falseStartIdentities:
    for i in range(numNucs):
        falseStartIdentities[key][i]=round(falseStartIdentities[key][i]/len(falseStartSpace),2)
falseStartFreqsDF=pd.DataFrame.from_dict(falseStartIdentities)
print(falseStartFreqsDF.T)

True start codon identities by position
      A    C    G    U
0     0  454    1   12
1     0  464    0    3
2     0  441    9   17
3     0    2    0  465
4     0    0    0  467
5     0    0    0  467
6   467    0    0    0
7   467    0    0    0
8     0  467    0    0
9     0  467    0    0
10  467    0    0    0
11    0    0    0  467
12    0    0  467    0
13    0    0    0  467
14    0  465    1    1
15    0    1    0  466
16    0    0  467    0
17    0    0  467    0
18    0    1  466    0
19  467    0    0    0
20    0   44    1  422
21  383    3   51   30
22    0  424    1   42
23    0    0    0  467
24    2   45    0  420

True start codon frequency by position
     0     1     2    3    4    5    6    7    8    9  ...    15   16   17  \
A  0.00  0.00  0.00  0.0  0.0  0.0  1.0  1.0  0.0  0.0 ...   0.0  0.0  0.0   
C  0.97  0.99  0.94  0.0  0.0  0.0  0.0  0.0  1.0  1.0 ...   0.0  0.0  0.0   
G  0.00  0.00  0.02  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  1.0  1.0   
U  0.03  0

In [9]:
matrix=startFreqsDF
matrix.to_pickle("loadData\\orf1matrix.pckl")
#score a sequence based on scoreing matrix
def scoreSequence(seq, matrix):
    positions=[0,1,2,3,4,5,6,7,8,9,13,14,15,16,17,18,19,20,21,22,23,24]
    weights=[3,3,1,0,1,1,4,4,2,3,0,0,0,1,1,2,2,2,4,2,1,1,2,2,2]
    seq=seq.upper()
    score=1
    for i in positions:
        try:
            positionScore=matrix[seq[i]][i]
        except:
            positionScore=0
        #for m in range(weights[i]):
        #    positionScore*=positionScore
        if positionScore==0:
            score*=.000001
        else:
            score*=positionScore
    return score

In [10]:
#true start scores
#print("True start Scores:")
trueHigh=0
trueLow=999
trueAVG=0
falseHigh=0
falseLow=999
falseAVG=0
#false start scores
print("False Start Scores:")
for i in range(len(falseStartSpace)):
    s=scoreSequence(falseStartSpace[i],matrix)
    #if s>.39: print(falseStartSpace[i] + str(s))
    if s>falseHigh: falseHigh=s
    if s<falseLow: falseLow=s
    falseAVG+=s
    #print(s)
falseAVG/=len(falseStartSpace)
print("True Start Scores: ")
for i in range(len(trueStartSpace)):
    s=scoreSequence(trueStartSpace[i],matrix)
    if s<falseHigh: print(trueStartSpace[i] + str(s))
    if s>trueHigh: trueHigh=s
    if s<trueLow: trueLow=s
    trueAVG+=s
    #print(s)
trueAVG/=len(trueStartSpace)

print("True High: " + str(trueHigh))
print("True Low: "+ str(trueLow))
print("True AVG: " + str(trueAVG))
print("False High: " + str(falseHigh))
print("False Low: " + str(falseLow))
print("False AVG: "+ str(falseAVG))

print(trueLow>falseHigh)
with open("loadData\\orf1scoreThreshold.txt",'w') as f:
    f.write(str((trueLow+falseHigh)/2))

False Start Scores:
True Start Scores: 
True High: 0.545600859804
True Low: 9.02682e-22
True AVG: 0.420425303699
False High: 7.3190359242e-38
False Low: 9.9e-122
False AVG: 7.74393225286e-43
True
